In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [4]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/DRM-QUAKESHELTER/DRM-QUAKESHELTER.csv", encoding='cp949')
print("원본 크기:", df.shape)
display(df.head())

# 2. 컬럼 이름 통일
df = df.rename(columns={"시군구명": "gu"})

# 3. 시설면적 수치형 변환
df["시설면적"] = pd.to_numeric(df["시설면적"], errors="coerce").fillna(0)

# 4. 구별 집계
agg = df.groupby("gu").agg(
    quakeshelter_count=("시설명", "count"),
    quakeshelter_total_area=("시설면적", "sum"),
    quakeshelter_avg_area=("시설면적", "mean")
).reset_index()

display(agg.head())

# 5. 점수화
scaler = MinMaxScaler(feature_range=(0,100))
scaled = scaler.fit_transform(agg[["quakeshelter_count",
                                   "quakeshelter_total_area",
                                   "quakeshelter_avg_area"]])

score_df = agg[["gu"]].copy()
for i, col in enumerate(["quakeshelter_count",
                         "quakeshelter_total_area",
                         "quakeshelter_avg_area"]):
    score_df[col+"_score"] = scaled[:, i]

# 6. 종합 점수 (평균)
score_df["quakeshelter_mean_score"] = score_df.filter(like="_score").mean(axis=1)

# 7. 저장
score_df.to_csv("../../result/DRM/DRM-QUAKESHELTER_result.csv", index=False, encoding="utf-8-sig")


score_df.head()

원본 크기: (1551, 11)


,피난처 ID,시도명,시군구명,시설명,상세주소,시설면적,경도,위도,구분,구분명,관리부서
0,1,서울특별시,동작구,동작고등학교 운동장,서울특별시 동작구 솔밭로 51(사당동),3246.0,126.9655,37.4820,1,지진실내구호소,070-4629-6407
1,2,서울특별시,동작구,동작초등학교 운동장,서울특별시 동작구 동작대로29길 214(사당동),2763.0,126.9768,37.4940,1,지진실내구호소,02-537-1773
2,3,서울특별시,도봉구,창림초등학교 운동장,서울특별시 도봉구 덕릉로63길 46(창동),2584.0,127.0417,37.6431,1,지진실내구호소,02-2091-5718
3,4,서울특별시,도봉구,창원초등학교 운동장,서울특별시 도봉구 해등로 103(창동),2704.0,127.0407,37.6521,1,지진실내구호소,02-2091-5779
4,5,서울특별시,도봉구,창일중학교 운동장,서울특별시 도봉구 노해로66길 60(창동),4800.0,127.0471,37.6484,1,지진실내구호소,02-2091-5678


,gu,quakeshelter_count,quakeshelter_total_area,quakeshelter_avg_area
0,강남구,120,875304.0,7294.200000
1,강동구,54,476576.0,8825.481481
2,강북구,51,141173.6,2768.109804
3,강서구,116,513310.0,4425.086207
4,관악구,80,230391.7,2879.896250


,gu,quakeshelter_count_score,quakeshelter_total_area_score,quakeshelter_avg_area_score,quakeshelter_mean_score
0,강남구,100.000000,100.000000,46.699850,82.233283
1,강동구,35.294118,49.159864,60.067502,48.173828
2,강북구,32.352941,6.394110,7.188371,15.311808
3,강서구,96.078431,53.843663,21.653304,57.191799
4,관악구,60.784314,17.769936,8.164235,28.906162
